In [2]:
from dotenv import dotenv_values
import os
import requests
import re
from langchain.callbacks import get_openai_callback

secrets = dotenv_values(r"..\..\langchain_pipeline\.env")
os.environ['OPENAI_API_KEY'] = secrets['openai_api']
os.environ['SERPAPI_API_KEY'] = secrets['serp_api_key']

In [5]:
openai_models_url = "https://api.openai.com/v1/models"

with get_openai_callback() as openai:
    openai_models = requests.get(openai_models_url, headers={"Authorization": "Bearer " + secrets['openai_api']}).json()
    print(openai)

Tokens Used: 0
	Prompt Tokens: 0
	Completion Tokens: 0
Successful Requests: 0
Total Cost (USD): $0.0


In [6]:
def find_models(all_models: dict, re_pattern: str = 'gpt'):
    gpt_models = [i['id'] for i in all_models['data'] if re.search(re_pattern, i['id'])]
    return gpt_models

In [7]:
find_models(openai_models, re_pattern='gpt')

['gpt-3.5-turbo-16k-0613',
 'gpt-3.5-turbo',
 'gpt-3.5-turbo-16k',
 'gpt-3.5-turbo-0613',
 'gpt-3.5-turbo-0301']

# Chat Model

In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

In [10]:
chat_llm = ChatOpenAI(temperature=0)

In [18]:
messages = [
    SystemMessage(content="You are a professional English and Dutch translator. You will be provided a prompt in either dutch or english, and you will have to answer the prompt and then provide a translation to Dutch or English."),
    HumanMessage(content="Tell me a sentence in dutch that an intermediate level person can understand.")
]

with get_openai_callback() as cb:
    response = chat_llm(messages)
    print(response.content)
    print(f"Total Tokens: {cb.total_tokens}")
    print(f"Prompt Tokens: {cb.prompt_tokens}")
    print(f"Completion Tokens: {cb.completion_tokens}")
    print(f"Total Cost (USD): ${cb.total_cost}")

Ik ga vanavond naar de bioscoop om een film te kijken.

Translation: I am going to the cinema tonight to watch a movie.
Total Tokens: 95
Prompt Tokens: 66
Completion Tokens: 29
Total Cost (USD): $0.00015700000000000002


# LLM Agents

In [4]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.llms import OpenAI

llm = OpenAI(temperature=0)
tools = load_tools(["serpapi", "llm-math"], llm=llm)
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [24]:
with get_openai_callback() as cb:
    response = agent.run(
        "What is a beagle that does not have hair?"
    )
    print(f"Total Tokens: {cb.total_tokens}")
    print(f"Prompt Tokens: {cb.prompt_tokens}")
    print(f"Completion Tokens: {cb.completion_tokens}")
    print(f"Total Cost (USD): ${cb.total_cost}")



> Entering new AgentExecutor chain...
 I need to find out what a beagle is and if it can have hair.
Action: Search
Action Input: "Beagle"
Observation: The beagle is a breed of small scent hound, similar in appearance to the much larger foxhound. The beagle was developed primarily for hunting hare, known as beagling.
Thought: I need to find out if a beagle can have hair.
Action: Search
Action Input: "Beagle hair"
Observation: Do Beagles have Hair or Fur? A well-debated question! Beagles have fur, although technically, hair and fur are the same thing (keratin), and on ...
Thought: I now know the final answer.
Final Answer: Beagles have fur, not hair.

> Finished chain.
Total Tokens: 858
Prompt Tokens: 784
Completion Tokens: 74
Total Cost (USD): $0.01716


# ChatModel Agent

In [18]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import MessagesPlaceholder
from langchain.memory import ConversationBufferMemory

MEMORY_KEY = 'chat_history'
memory = ConversationBufferMemory(memory_key=MEMORY_KEY, return_messages=True)

chat_model = ChatOpenAI(temperature=0)
tools = load_tools(["serpapi"])
agent = initialize_agent(tools, chat_model, agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION, verbose=True, memory=memory, handle_parsing_errors=True)

In [ ]:
with get_openai_callback() as cb:
    response = agent.run(
        "What is a beagle that does not have hair?"
    )
    print(f"Total Tokens: {cb.total_tokens}")
    print(f"Prompt Tokens: {cb.prompt_tokens}")
    print(f"Completion Tokens: {cb.completion_tokens}")
    print(f"Total Cost (USD): ${cb.total_cost}")

In [19]:
with get_openai_callback() as cb:
    print(agent.run(
        "Hi, my name is Lian, and I am testing your memory."
    ))
    print(cb)



> Entering new AgentExecutor chain...
{
    "action": "Final Answer",
    "action_input": "Nice to meet you, Lian! How can I assist you today?"
}

> Finished chain.
Nice to meet you, Lian! How can I assist you today?
Tokens Used: 521
	Prompt Tokens: 490
	Completion Tokens: 31
Successful Requests: 1
Total Cost (USD): $0.000797


In [20]:
print(agent.run(input="I want you to come up with a nickname for me."))



> Entering new AgentExecutor chain...
Could not parse LLM output: Sure, I'd be happy to come up with a nickname for you! Could you please provide me with some information about yourself? For example, what are some of your interests or characteristics that you would like the nickname to reflect?
Observation: Invalid or incomplete response
Thought:{
    "action": "Final Answer",
    "action_input": "Sure, I'd be happy to come up with a nickname for you! Could you please provide me with some information about yourself? For example, what are some of your interests or characteristics that you would like the nickname to reflect?"
}

> Finished chain.
Sure, I'd be happy to come up with a nickname for you! Could you please provide me with some information about yourself? For example, what are some of your interests or characteristics that you would like the nickname to reflect?
